In [3]:
%load_ext autoreload
%autoreload 2
main_dir = r'../../..'
data_path = main_dir+'/data'
import sys
sys.path.append(main_dir)
from typing import List, Set, Dict, Optional, Any, Tuple, Type, Union

from BayDS.lib.pipeline import *
from BayDS.lib.io import *
import os


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
data_dir = f'{main_dir}/Snapshots/1/06-Combine'
p = Pipeline(working_folder=f'{main_dir}/Snapshots/1/07-Untitled')

p.add_node(LoaderNode, None, 'data',
           params={
               'input_directory': data_dir,
               'file': 'data.pkl'
           })
p.add_node(LoaderNode, None, 'numerical_columns',
           params={
               'input_directory': data_dir,
               'file': 'numerical_columns.yaml'
           })
p.add_node(LoaderNode, None, 'categorical_columns',
           params={
               'input_directory': data_dir,
               'file': 'categorical_columns.yaml'
           })
p.save()
p.run(verbose=True)
# p.save_data()

---------------------------
0: LoaderNode [2019-09-21 21:15:48]
params:
 {'input_directory': '../../../Snapshots/1/06-Combine', 'file': 'data.pkl'}
---------------------------
1: LoaderNode [2019-09-21 21:15:51]
params:
 {'input_directory': '../../../Snapshots/1/06-Combine', 'file': 'numerical_columns.yaml'}
---------------------------
2: LoaderNode [2019-09-21 21:15:51]
params:
 {'input_directory': '../../../Snapshots/1/06-Combine', 'file': 'categorical_columns.yaml'}


In [8]:
p.add_node(DownsamplingTrainNode, 'data', 'data',
           params={
           })

p.save()
p.run(verbose=True)


---------------------------
3: DownsamplingTrainNode [2019-09-21 21:21:48]
params:
 {}


In [12]:
df = p.data['data']

In [28]:
from sklearn.feature_selection import RFECV

X = df.sort_values('TransactionDT').drop(['isFraud', 'Date'], axis=1)
y = df.sort_values('TransactionDT')['isFraud']

X.fillna(-999, inplace=True)
X.replace(np.inf, np.nan, inplace=True)
X.replace(-np.inf, np.nan, inplace=True)
X['screen_width'] = X['screen_width'].astype(np.float16)
X['screen_height'] = X['screen_height'].astype(np.float16)

ValueError: could not convert string to float: 'None'

In [ ]:
for col in X.columns:
    print(col,X[col].dtype)

In [16]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 0,
          'n_jobs': -1
         }

In [21]:
import lightgbm as lgb
from sklearn.model_selection import KFold
clf = lgb.LGBMClassifier(**params)
rfe = RFECV(estimator=clf, step=10, cv=KFold(n_splits=5, shuffle=False), scoring='roc_auc', verbose=2)

In [22]:
rfe.fit(X, y)

ValueError: could not convert string to float: 'None'